<a href="https://colab.research.google.com/github/Showmick119/Samay/blob/main/moment_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Dependencies**

In [ ]:
!pip install samay-0.1.0-cp311-cp311-linux_x86_64.whl

In [ ]:
!pip install --upgrade -U numpy --force

# **Loading Moment Model**

In [ ]:
from samay.model import MomentModel
from samay.dataset import MomentDataset
from samay.utils import load_args

repo = "AutonLab/MOMENT-1-large"
config = {
    "task_name": "forecasting",
    "forecast_horizon": 192,
    "head_dropout": 0.1,
    "weight_decay": 0,
    "freeze_encoder": True,  # Freeze the patch embedding layer
    "freeze_embedder": True,  # Freeze the transformer encoder
    "freeze_head": False,  # The linear forecasting head must be trained
}
mmt = MomentModel(config=config, repo=repo)

# **Loading Dataset**

In [ ]:
!wget https://raw.githubusercontent.com/kage08/Samay/refs/heads/main/data/data/ETTh1.csv

# **Finetune Moment Model on the ETT dataset**

In [ ]:
train_dataset = MomentDataset(name="ett", datetime_col='date', path='../src/tsfmproject/models/moment/data/ETTh1.csv',
                              mode='train', horizon_len=192)
# dtl = train_dataset.get_data_loader()

val_dataset = MomentDataset(name="ett", datetime_col='date', path='../src/tsfmproject/models/moment/data/ETTh1.csv',
                            mode='test', horizon_len=192)
# path = '../src/tsfmproject/models/moment/data/ETTh1.csv'

# dataset = MomentDataset(name="ett", datetime_col='date', path=path,
#                               mode='train', horizon=192)

finetuned_model = mmt.finetune(train_dataset, task_name="forecasting")
mmt.evaluate(val_dataset, task_name="forecasting")

# **Test the Finetuned Model**

In [ ]:
# avg_loss, trues, preds, histories = mmt.evaluate(val_dataset, task_name='forecasting')
# print("Validation loss:", avg_loss)
mmt.plot(val_dataset, task_name='forecasting')

# **Visualization of the evaluation**

In [ ]:
# import matplotlib.pyplot as plt

# # Pick a random channel and time index
# trues = np.array(trues)
# preds = np.array(preds)
# histories = np.array(histories)
# channel_idx = np.random.randint(0, 7)
# time_index = np.random.randint(0, trues.shape[0])

# history = histories[time_index, channel_idx, :]
# true = trues[time_index, channel_idx, :]
# pred = preds[time_index, channel_idx, :]

# plt.figure(figsize=(12, 4))

# # Plotting the first time series from history
# plt.plot(range(len(history)), history, label='History (512 timesteps)', c='darkblue')

# # Plotting ground truth and prediction
# num_forecasts = len(true)

# offset = len(history)
# plt.plot(range(offset, offset + len(true)), true, label='Ground Truth (192 timesteps)', color='darkblue', linestyle='--', alpha=0.5)
# plt.plot(range(offset, offset + len(pred)), pred, label='Forecast (192 timesteps)', color='red', linestyle='--')

# plt.title(f"ETTh1 (Hourly) -- (idx={time_index}, channel={channel_idx})", fontsize=18)
# plt.xlabel('Time', fontsize=14)
# plt.ylabel('Value', fontsize=14)
# plt.legend(fontsize=14)
# plt.show()